# Try to train a model using CNN_1_P300_PAMI

In [1]:
import os
os.chdir("../")

In [2]:
DATA_DIR = "./data/raw/BCI_Comp_III_Wads_2004/"
SECONDS_TO_SLICE = 0.65
FS = 240
NUM_ELECTRODES = 64
RANDOM_STATE = 42

In [3]:
from src.data import preprocessing,extract_data
from src.models import model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_STATE)


Using TensorFlow backend.


In [4]:
A_train = extract_data.extract_data(DATA_DIR+"Subject_A_Train.mat",SECONDS_TO_SLICE)
A_test = extract_data.extract_data(DATA_DIR+"Subject_A_Test.mat",SECONDS_TO_SLICE)
print(A_train.keys())
print(A_test.keys())

print(A_train['signal'].shape)
print(A_test['signal'].shape)
print(A_train['code'].shape)
print(A_test['code'].shape)
print(A_train['label'].shape)
print(A_train['targetchar'].shape)

dict_keys(['targetchar', 'code', 'label', 'signal'])
dict_keys(['code', 'signal'])
(85, 15, 12, 64, 156)
(100, 15, 12, 64, 156)
(85, 15, 12)
(100, 15, 12)
(85, 15, 12)
(1,)


In [5]:
A_train['signal'] = preprocessing.subsample(A_train['signal'],2)
A_test['signal'] = preprocessing.subsample(A_test['signal'],2)

A_train['signal'] = preprocessing.butter_bandpass_filter(A_train['signal'],lowcut = 0.1,highcut = 20,
                                                         fs = int(FS/2),order = 5,axis = -1)
A_test['signal'] = preprocessing.butter_bandpass_filter(A_test['signal'],lowcut = 0.1,highcut = 20,
                                                         fs = int(FS/2),order = 5,axis = -1)

In [6]:
A_train_mean = A_train['signal'].mean(axis = -1,keepdims = True)
A_train_std = A_train['signal'].std(axis = -1,keepdims = True)
A_test_mean = A_test['signal'].mean(axis = -1,keepdims = True)
A_test_std = A_test['signal'].std(axis = -1,keepdims = True)

A_train['signal'] = (A_train['signal']-A_train_mean)/A_train_std
A_test['signal']  = (A_test['signal']-A_test_mean)/A_test_std

## Stratified sampling

In [7]:
X_train = A_train['signal'].reshape([-1,A_train['signal'].shape[-2],A_train['signal'].shape[-1]])
y_train = A_train['label'].reshape([-1])


In [8]:
print(X_train.shape)
print(y_train.shape)

(15300, 64, 78)
(15300,)


In [9]:
X_train = np.expand_dims(X_train, axis = -1)

In [10]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, 2)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X_train, y_train, test_size = 0.05, random_state = RANDOM_STATE)

In [12]:
print(X_train.shape)
print(X_val.shape)

(14535, 64, 78, 1)
(765, 64, 78, 1)


In [13]:
model_pami = model.CNN_1_P300_PAMI_BCIIII(Ns = 10, seconds_to_slice=0.65)

In [15]:
from keras.callbacks import EarlyStopping

In [24]:
earlystopping = EarlyStopping(monitor = 'val_acc', patience = 10)

In [25]:
model_pami.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['acc', 'mse'])
model_pami.fit(x = X_train, y = y_train, batch_size=32, epochs = 1000, callbacks = [earlystopping], 
              validation_data = [X_val,y_val])

Train on 14535 samples, validate on 765 samples
Epoch 1/1000
14535/14535 [==============================] - 4s - loss: 0.4284 - acc: 0.8337 - mean_squared_error: 0.1325 - val_loss: 0.4483 - val_acc: 0.8261 - val_mean_squared_error: 0.1397
Epoch 2/1000
14535/14535 [==============================] - 4s - loss: 0.4225 - acc: 0.8337 - mean_squared_error: 0.1307 - val_loss: 0.4448 - val_acc: 0.8261 - val_mean_squared_error: 0.1387
Epoch 3/1000
14535/14535 [==============================] - 4s - loss: 0.4152 - acc: 0.8337 - mean_squared_error: 0.1284 - val_loss: 0.4407 - val_acc: 0.8261 - val_mean_squared_error: 0.1375
Epoch 4/1000
14535/14535 [==============================] - 4s - loss: 0.4074 - acc: 0.8336 - mean_squared_error: 0.1261 - val_loss: 0.4360 - val_acc: 0.8281 - val_mean_squared_error: 0.1360
Epoch 5/1000
14535/14535 [==============================] - 4s - loss: 0.4002 - acc: 0.8346 - mean_squared_error: 0.1239 - val_loss: 0.4334 - val_acc: 0.8255 - val_mean_squared_error: 0.13

In [26]:
X_pred = model_pami.predict(X_val)

In [27]:
X_pred.shape

(765, 2)

In [32]:
from src.metrics import metrics
metrics.print_all_stats(np.argmax(y_val,axis=-1),np.argmax(X_pred, axis = -1))

TP:30	TN:613	FP:19	FN:103

Reco:0.8405228758169935	Recall:0.22556390977443608	Precision:0.6122448979591837	Silence:0.7744360902255639

Noise:0.3877551020408163	Error:0.9172932330827067	F-measure:2.0


In [30]:
np.argmax(X_pred,axis=-1)

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [31]:
y_val

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [33]:
earlystopping = EarlyStopping(monitor = 'val_mean_squared_error', patience = 2)
model_pami.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['acc', 'mse'])
model_pami.fit(x = X_train, y = y_train, batch_size=32, epochs = 5, callbacks = [earlystopping], 
              validation_data = [X_val,y_val])

Train on 14535 samples, validate on 765 samples
Epoch 1/5
14535/14535 [==============================] - 5s - loss: 0.3436 - acc: 0.8592 - mean_squared_error: 0.1049 - val_loss: 0.3928 - val_acc: 0.8301 - val_mean_squared_error: 0.1229
Epoch 2/5
14535/14535 [==============================] - 4s - loss: 0.3431 - acc: 0.8603 - mean_squared_error: 0.1047 - val_loss: 0.3923 - val_acc: 0.8438 - val_mean_squared_error: 0.1222
Epoch 3/5
14535/14535 [==============================] - 4s - loss: 0.3425 - acc: 0.8598 - mean_squared_error: 0.1044 - val_loss: 0.3926 - val_acc: 0.8405 - val_mean_squared_error: 0.1223
Epoch 4/5
14535/14535 [==============================] - 4s - loss: 0.3419 - acc: 0.8593 - mean_squared_error: 0.1044 - val_loss: 0.3943 - val_acc: 0.8314 - val_mean_squared_error: 0.1233
Epoch 5/5
14535/14535 [==============================] - 5s - loss: 0.3412 - acc: 0.8609 - mean_squared_error: 0.1041 - val_loss: 0.3925 - val_acc: 0.8366 - val_mean_squared_error: 0.1227
